## Runnable Interface 介绍与使用

为了尽可能简化创建自定义链的过程，Langchain 实现了一个 **[Runnable](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)** 协议。

许多 LangChain 组件都实现了 Runnable 协议，包括 chat models, LLMs, output parsers, retrievers, prompt templates等等。此外，还有几个用于处理可运行对象的[有用原语](https://python.langchain.com/v0.1/docs/expression_language/primitives/)。

Runnable 是一个标准接口，包括：

- stream：流式返回生成内容（chunk）
- invoke：对输入调用该链
- batch：对输入列表调用该链

不同组件的输入和输出类型有所差异:

| 组件    | 输入类型                                           | 输出类型           |
| ------------ | ----------------------------------------------------- | --------------------- |
| Prompt       | Dictionary                                            | PromptValue           |
| ChatModel    | Single string, list of chat messages or a PromptValue | ChatMessage           |
| LLM          | Single string, list of chat messages or a PromptValue | String                |
| OutputParser | The output of an LLM or ChatModel                     | Depends on the parser |
| Retriever    | Single string                                         | List of Documents     |
| Tool         | Single string or dictionary, depending on the tool    | Depends on the tool   |


所有 Runnable 对象都显式描述输入和输出 Schema，以检查输入和输出格式：

- input_schema：从Runnable的结构自动生成的输入Pydantic模型
- output_schema：从Runnable的结构自动生成的输出Pydantic模型 


### Input Schema

为了演示如何使用，下面我们创建一个超级简单的PromptTemplate + ChatModel链。

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("讲个关于 {topic} 的笑话吧")
chain = prompt | model

#### schema 方法

一个描述 Runnable 接受的输入的说明。这是根据任何 Runnable 结构动态生成的 Pydantic 模型。您可以调用 .schema() 来获取 `JSONSchema` 表示。

In [2]:
# 查看 Chain 的输入类型
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic']}

In [3]:
# 查看 Prompt 的输入类型（Chain的输入从 Prompt 开始，因此输入类型一致）
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic']}

In [4]:
# 查看 Model 的输入类型
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {

### Output Schema

输出类型仍然可以调用 .schema() 来获取其 `JSONSchema` 表示。

In [5]:
# 查看 Chain 的输出类型
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'title': 'Id', 'type': 'string'},
    'type': {'title': 'Type', 'enum': ['tool_call'], 'type': 'string'}},
   'required': ['name', 'args', 'id']},
  'InvalidToolCall': {'title': 'InvalidToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'error': {'title': 'Error', 'type': 'string'},
    'type': {'title': 'Type',
     'enum': ['invalid_tool_call'],
     'type': 'string'}

### Stream

使用 .stream() 方法查看（同步）流式输出结果

In [6]:
for s in chain.stream({"topic": "程序员"}):
    print(s.content, end="", flush=True)

当然可以！这里有一个关于程序员的笑话：

有一天，一位程序员走进了酒吧，点了一杯酒。酒保问他：“你想要加冰吗？” 

程序员回答：“不，我只想要一个整数。” 

酒保困惑地问：“什么是一个整数？” 

程序员微笑着说：“就是不加冰的。” 

希望这个笑话能让你笑一笑！

### Invoke
使用 .invoke() 方法单次（同步）调用

In [7]:
chain.invoke({"topic": "程序员"})

AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，一个程序员走进一家咖啡馆，点了一杯咖啡。服务员问他：“要加糖吗？”\n\n程序员回答说：“不，我只想要一个‘0’或‘1’。”\n\n服务员困惑地问：“这是什么意思？”\n\n程序员笑着说：“因为我只需要‘甜’或者‘不甜’！” \n\n希望这个笑话能让你笑一笑！', response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 18, 'total_tokens': 120}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_9722793223', 'finish_reason': 'stop', 'logprobs': None}, id='run-e0cc952f-47ee-49c1-82c4-711145e26be2-0')

### Batch
使用 .batch() 方法（同步）批量调用

In [8]:
chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[AIMessage(content='当然可以！这里有一个关于程序员的笑话：\n\n有一天，程序员走进一家酒吧，点了一杯啤酒。 bartender问他：“你要喝什么类型的啤酒？”程序员回答：“我想要一杯‘Bug-Free’的啤酒！” bartender愣了一下，问道：“那是什么样的啤酒？”程序员笑着说：“当然是一种不会出现任何问题的啤酒，但我知道这绝对不可能！”\n\n希望你喜欢这个笑话！', response_metadata={'token_usage': {'completion_tokens': 113, 'prompt_tokens': 18, 'total_tokens': 131}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_9722793223', 'finish_reason': 'stop', 'logprobs': None}, id='run-de7e977b-ec6a-4e62-bb91-40d8b6d76fa2-0'),
 AIMessage(content='当然可以！这里有一个关于产品经理的笑话：\n\n有一天，产品经理、开发者和设计师一起去参加一个团队建设活动。在活动中，他们需要一起搭建一个帐篷。\n\n产品经理说：“我们需要先讨论一下用户需求，确保帐篷能满足所有用户的期望。”\n\n开发者说：“好吧，但我们必须先确定技术可行性，看看能不能在这个预算内搭建。”\n\n设计师则说：“等等，我觉得我们应该先选一个更好看的颜色！”\n\n最后，帐篷还是没搭好，但他们却在讨论中达成了共识：下次一定要先搭建一个“最小可行产品”！  \n\n希望你喜欢这个笑话！', response_metadata={'token_usage': {'completion_tokens': 157, 'prompt_tokens': 16, 'total_tokens': 173}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_5bd87c427a', 'finish_reason': 'stop', 'logprobs': None}, id='run-7267180f-a443-4979-a0a7-f3d5ae70304

In [9]:
messages = chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

In [10]:
# 使用 StrOutputParser 来处理 Batch 批量输出
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

for idx, m in enumerate(messages):
    print(f"笑话{idx}:\n")
    print(output_parser.invoke(m))
    print("\n")

笑话0:

当然可以！这是一个关于程序员的笑话：

有一天，一个程序员走进酒吧，看到了一张空桌子。他对酒保说：“请给我一杯水。”  
酒保点点头，转身去倒水。  
程序员等了好久，酒保还是没有回来。于是他又说：“请再给我一杯水。”  
酒保依然没有回应。  
最后，程序员忍不住了，站起来大喊：“如果你不来，我就要自己写一个水的获取函数！”

希望这个笑话能让你笑一笑！


笑话1:

当然可以！这是一个关于产品经理的笑话：

有一天，产品经理、开发人员和设计师一起坐在会议室里讨论新产品的功能。

产品经理说：“我们需要添加一个用户反馈的功能，这样我们可以收集用户的意见。”

开发人员皱了皱眉头：“可是这会增加我们的开发时间。”

设计师插嘴说：“我觉得用户根本不知道他们想要什么。”

产品经理微笑着说：“没关系，我们可以先把这个功能做出来，然后再问他们。”

开发人员和设计师对视一眼，异口同声地说：“这听起来像是你们的工作！”

希望这个笑话让你开心！


笑话2:

当然可以！这里有一个关于测试经理的笑话：

有一天，一位测试经理走进咖啡店，点了一杯咖啡。咖啡师问他：“您想要加糖吗？”

测试经理回答：“请先给我一杯不加糖的咖啡，然后我会测试一下加糖的效果！”

咖啡师无奈地摇了摇头：“您真是个彻底的测试经理，连喝咖啡也要进行AB测试！”

希望这个笑话能让你开心！




## 异步操作

这些方法也有相应的异步方法，应与 `asyncio` 的 `await` 语法一起使用以进行并发操作：

- astream：异步地流式返回生成内容（chunk）
- ainvoke：异步地对输入调用该链
- abatch：异步地对输入列表调用该链
- astream_log: 在发生时会返回中间步骤，并且最终返回结果之外。
- astream_events: beta 流式传输事件，在 langchain-core 0.1.14 中引入


### Async Stream

In [11]:
async for s in chain.astream({"topic": "程序员"}):
    print(s.content, end="", flush=True)

当然可以！这是一个关于程序员的笑话：

为什么程序员总是喜欢在海边工作？

因为那里有很多“海量数据”！  

希望你喜欢这个笑话！如果还想听更多，随时告诉我！

### Async Invoke

In [12]:
await chain.ainvoke({"topic": "程序员"})

AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n为什么程序员总是喜欢在海边工作？\n\n因为他们总是想在“代码”中找到“海洋”！ \n\n（“海洋”在这里代表“大量的数据”）希望你喜欢这个笑话！', response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 18, 'total_tokens': 78}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-b84ea289-a442-48d6-9319-969535f23133-0')

### Async Batch

In [13]:
await chain.abatch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，程序员去买水果。他走到水果摊前，看见老板在称西瓜。\n\n程序员问：“这个西瓜多少钱？”\n\n老板回答：“每个10块。”\n\n程序员思考了一下，问：“那如果我买两个，能不能给我打个折？”\n\n老板笑了笑：“当然可以，两个20块！”\n\n程序员愣了一下，接着说：“我明白了，您是只支持‘零和’交易的！”\n\n老板一脸懵：“什么是‘零和’交易？”\n\n程序员微微一笑：“就是我买两个，您就没得赚了！”\n\n希望这个笑话能让你开心！', response_metadata={'token_usage': {'completion_tokens': 156, 'prompt_tokens': 18, 'total_tokens': 174}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-b05d03a7-de24-40cb-b127-bcc7557a9bcc-0'),
 AIMessage(content='当然可以！这是一个关于产品经理的笑话：\n\n有一天，一个产品经理走进一家咖啡店，点了一杯咖啡。店员问他：“要加糖吗？” \n\n产品经理回答：“请把这个选项放到用户调查中再决定。” \n\n店员无奈地说：“好吧，那我给你一杯无糖的咖啡。” \n\n产品经理愣了一下：“等等，这样用户就没有选择了！” \n\n店员笑着说：“没关系，反正他们也喝不到！” \n\n希望这个笑话能让你开心！', response_metadata={'token_usage': {'completion_tokens': 127, 'prompt_tokens': 16, 'total_tokens': 143}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-d10cb1d9-8a7f-4e91-92b6-1